# Neuromap Project Neurohackademy 2019

In [1]:
# install libraries/ set up script
import pandas as pd
import numpy as np
import folium
import geopy
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="neurohackmap")
#! pip install ipyleaflet 
import ipyleaflet as lf
from ipyleaflet import (Map, basemaps, basemap_to_tiles, Circle, CircleMarker, LayersControl,
                        FullScreenControl, WidgetControl, Marker, Popup)
from ipywidgets import (IntSlider, ColorPicker, jslink, HTML)

In [2]:
# sets the defaults for viewing the dataframe in the jupyter notebook
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [3]:
# define missing values
missing_values = ["  ", "-"]

#load in the questionnaire response data from a csv file 
df = pd.read_csv('NeuroMap 2.csv', na_values = missing_values, encoding='latin-1')

#optionally view the data
#df.head(20)

In [4]:
## data cleaning steps 
## rename column questions to shorter
df.columns = ['timestamp','consent','name','birthplace','birth_year', 'gender',
              'undergrad_deg','undergrad_loc',  'undergrad_inst','undergrad_research',
              'ra_qual','ra_lm_loc', 'ra_lm_inst', 'ra_lm_research',
              'masters_qual','masters_loc', 'masters_inst', 'masters_research', 
              'phd_qual','phd_loc', 'phd_inst', 'phd_research',
              'post_doc_qual', 'postdoc_loc' ,'postdoc_inst','postdoc_research', 
            'faculty_qual', 'faculty_loc', 'faculty_inst', 'faculty_research',
              'google_scholar']

## during code development, irregularities in input style that caused geopy errors were identified.
## these have been manually corrected as per below
replace_dict_birth = {
    "Tijuana, Baja California, MÌ©xico": "Tijuana, Baja California, Mexico",
    "Ìävreux, Normandy, France": "Normandy, France"
}
df['birthplace'] = df['birthplace'].replace(replace_dict_birth) 


replace_dict_undergrad = {
    'Tijuana, Baja California, MÌ©xico': 'Tijuana, Baja California, Mexico',
    'Raleigh & Chapel Hill, North Carolina, USA': 'Chapel Hill, North Carolina, USA'
}
df['undergrad_loc'] = df['undergrad_loc'].replace(replace_dict_undergrad) 

replace_dict_research = {
    'Central Institute of Chemistry and Mechanics': 'Nagatinskaya, Moscow, Russia',
    'Boston, MA & San Francisco, CA': 'San Francisco, CA'    
}

df['ra_lm_loc'] = df['ra_lm_loc'].replace(replace_dict_research)

replace_dict_phd = {
    'Rio de Janeiro, Brazil AND Montreal, Canada': 'Rio de Janeiro, Brazil',
    'Italy/Japan':'Italy'
}
df['phd_loc'] = df['phd_loc'].replace(replace_dict_phd) 


replace_dict_masters = {
    'Neuroscience': 'Strasbourg, Alsace, France',
    'Mexico city, MÌ©xico': 'Mexico city, Mexico'
}
df['masters_loc'] = df['masters_loc'].replace(replace_dict_masters)


replace_dict_postdoc = {
    'New Haven, CT, United States AND Durham, NC, United States': 'New Haven, CT, United States',
    '1) York, UK; 2) Sussex, UK': 'York, UK',
    'Seattle, WA and Cambridge, MA':'Seattle, WA'
}
df['postdoc_loc'] = df['postdoc_loc'].replace(replace_dict_postdoc)


In [5]:
# The following code uses geopy to city, state, country information to get latitude and longitude
# This is done for birth_location, undergrad_location, research position location, PhD location and 
# post doc location

# Birth_location is done first
# First create empty structures for your lat and long values
lat_list = []
long_list = []

#loop through birthplace locations
for birth_location in df['birthplace']:
    #print(location_1)
    
    if pd.isnull(birth_location):
        lat = None
        long = None #this section is to manage missing data
    else:
        location_1 = geolocator.geocode(birth_location, timeout=10)
        if location_1 is None:
            raise ValueError("Geocode failed") # this section is to manage data in wrong format
        lat = location_1.latitude
        long = location_1.longitude
    
    lat_list.append(lat)
    long_list.append(long)
    #print((lat,long,birth_location))
        
#Saving lat and long in separate columns in the dataframe    
df['HometownLatitude'] = lat_list
df['HometownLatitude'] = df['HometownLatitude'].astype('float')

df['HometownLongitude'] = long_list
df['HometownLongitude'] = df['HometownLongitude'].astype('float')
df.head(20)

#Get the column with hometown location
locs = df.loc[:,"birthplace"]

#Plot hometown locations on the map!
birth_map = folium.Map()

#Loop through locations and add the markers on the map
for home_location in range(len(locs)): 
    folium.Marker([lat_list[home_location], long_list[home_location]], popup=locs[home_location]).add_to(birth_map)
    
# output the map
#birth_map


In [6]:
#Check your dataframe  
#df.head(60)

In [7]:
# Undergrad location is done second
# First create empty structures for your lat and long values
lat_list = []
long_list = []

#loop through undergrad locations
for undergrad_location in df['undergrad_loc']:
    #print(location_1)
    
    if pd.isnull(undergrad_location):
        lat = None
        long = None
    else:
        location_1 = geolocator.geocode(undergrad_location, timeout=10)
        if location_1 is None:
            raise ValueError("Geocode failed")
        lat = location_1.latitude
        long = location_1.longitude
    
    lat_list.append(lat)
    long_list.append(long)
    #print((lat,long,undergrad_location))
        
# #Saving lat and long in separate columns in the original dataframe    
df['UndergradLatitude' ] = lat_list
df['UndergradLatitude'] = df['UndergradLatitude'].astype('float')

df['UndergradLongitude'] = long_list
df['UndergradLongitude'] = df['UndergradLongitude'].astype('float')

# because there are missing values which mean the mapping won't work you need to create a subset df

#create a subset of df
df_undergrad = df.loc[:,"UndergradLatitude":"UndergradLongitude"] 
df_undergrad = df_undergrad.dropna()

# get undergrad locations 
undergrad_locs = df.loc[:,"undergrad_loc"]
#print(df_undergrad)

#modify lat_list and long_list to remove missing values
lat_list = list(filter(None, lat_list))
long_list = list(filter(None, long_list))
undergrad_locs = [x for x in undergrad_locs if pd.notna(x)]

# check formats are correct with the NA values removed
print(lat_list)
print(long_list)
print(undergrad_locs)
type(undergrad_locs)

# undergrad locations plotting 
undergrad_map = folium.Map()
#Loop through locations and add the markers on the map
for undergrad_location in range(len(undergrad_locs)): 
    folium.Marker([lat_list[undergrad_location], long_list[undergrad_location]],
                  popup=undergrad_locs[undergrad_location]).add_to(undergrad_map)
    
#display map
#undergrad_map

[41.8239891, 45.886548, -34.4243941, 22.2793278, 42.2681569, 38.9719384, -34.6075616, 55.7504461, 18.9387711, 37.4443293, 40.4258686, 32.527002, 25.0375198, 29.8693496, 42.0128695, 37.5666791, 32.5010188, 52.1518157, 36.1556805, 33.5778631, 34.0536909, 33.7872568, -22.9110137, 45.421106, 31.778345, 35.9131542, 55.0282171, 29.9499323, 43.0821793, 35.7006177, 40.7127281, 38.8339578, 43.653963, 31.778345, 43.157285, 34.0966764, 51.4893335, 42.3602534, 43.703622, 51.4538022, 49.4404591, 43.6307863, 52.2646577, 52.9534161, 25.7742658, 52.1518157, 34.1476452, 41.5623178, 1.3408528, 52.266837, 42.6384261, 53.0758196, 39.2847494, 38.8950092]
[-71.4128343, 11.0452369, 150.89385, 114.1628131, -83.7312291, -95.2359496, -58.437076, 37.6174943, 72.8353355, -122.1598465, -86.9080655, -85.4367484053398, 121.5636796, 77.8902124, -73.9081901, 126.9782914, -116.9646629, 4.48110886662043, -95.9929113, -101.8551665, -118.2427666, -117.8503088, -43.2093727, -75.690308, 35.2250786, -79.05578, 82.9234509, -9

In [8]:
# Research assistant and lab manager position locations 
# First create empty structures for your lat and long values
lat_list = []
long_list = []

# Loop through rows to get the longitude and latitude of the RA_locations (be aware of missing data)
for research_location in df['ra_lm_loc']:
    #print(location_1)
    
    if pd.isnull(research_location):
        lat = None
        long = None
    else:
        location_1 = geolocator.geocode(research_location, timeout=10)
        if location_1 is None:
            raise ValueError("Geocode failed")
        lat = location_1.latitude
        long = location_1.longitude
    
    lat_list.append(lat)
    long_list.append(long)
    #print((lat,long,research_location))
        
# #Saving lat and long in separate columns in the dataframe    
df['RA_LM_Latitude' ] = lat_list
df['RA_LM_Latitude'] = df['RA_LM_Latitude'].astype('float')

df['RA_LM_Longitude'] = long_list
df['RA_LM_Longitude'] = df['RA_LM_Longitude'].astype('float')
#df.head(40)

# create a map with research position locations
df_research = df.loc[:,"RA_LM_Latitude":"RA_LM_Longitude"] #create a subset of df to deal with the na problem
df_research = df_research.dropna()
research_locs = df.loc[:,"ra_lm_loc"]
#print(df_undergrad)

lat_list = list(filter(None, lat_list))
long_list = list(filter(None, long_list))
research_locs = [x for x in research_locs if pd.notna(x)]

# check formats are correct with the NA values removed
print(lat_list)
print(long_list)
print(research_locs)
type(research_locs)

# research locations plotting 
research_map = folium.Map()
#Loop through locations and add the markers on the map
for research_location in range(len(research_locs)): 
    folium.Marker([lat_list[research_location], long_list[research_location]],
                  popup=research_locs[research_location]).add_to(research_map)

#display map
#research_map

[41.3082138, -37.8142176, 52.4775396, 39.9527237, -34.6075616, 55.6828925, 39.9527237, 40.7127281, 25.0375198, 40.7127281, 37.4443293, 45.5202471, 37.8708393, 37.7792808, 35.7006177, 40.4416941, 38.8950092, 52.5170365, 34.0536909, 37.7792808, 42.3602534, 38.9848265, 25.7742658, 40.7127281, 40.4416941]
[-72.9250518, 144.9631608, -1.894053, -75.1635262, -58.437076, 37.6223775, -75.1635262, -74.0060152, 121.5636796, -74.0060152, -122.1598465, -122.6741949, -122.2728639, -122.4192363, 51.4013785, -79.9900861, -77.0365625, 13.3888599, -118.2427666, -122.4192363, -71.0582912, -77.0946459, -80.1936589, -74.0060152, -79.9900861]
['New Haven, CT, USA', 'Melbourne, Victoria, Australia', 'Birmingham, UK', 'Philadelphia, PA, USA', 'Buenos Aires, Argentina', 'Nagatinskaya, Moscow, Russia', 'Philadelphia, USA', 'New York City, New York, USA', 'Taipei, Taiwan', 'New York, NY', 'Palo Alto, California, USA', 'Portland, OR, USA', 'Berkeley, CA, USA', 'San Francisco, CA, USA', 'Tehran, Tehran, Iran', 'Pi

In [9]:
# PhD position locations 
# create empty structures
lat_list = []
long_list = []
for phd_location in df['phd_loc']:
    #print(location_1)
    
# Loop through rows to get the longitude and latitude of the RA_locations (be aware of missing data)    
    if pd.isnull(phd_location):
        lat = None
        long = None
    else:
        location_1 = geolocator.geocode(phd_location, timeout=10)
        if location_1 is None:
            raise ValueError("Geocode failed")
        lat = location_1.latitude
        long = location_1.longitude
    
    lat_list.append(lat)
    long_list.append(long)
    #print((lat,long,phd_location))
        
# #Saving lat and long in separate columns in the dataframe    
df['PHD_Latitude' ] = lat_list
df['PHD_Latitude'] = df['PHD_Latitude'].astype('float')

df['PHD_Longitude'] = long_list
df['PHD_Longitude'] = df['PHD_Longitude'].astype('float')

#create a subset of df to deal with the na problem
df_phd = df.loc[:,"PHD_Latitude":"PHD_Longitude"] 
df_phd = df_phd.dropna()
phd_locs = df.loc[:,"phd_loc"]
#print(df_undergrad)

#remove NA values
lat_list = list(filter(None, lat_list))
long_list = list(filter(None, long_list))
phd_locs = [x for x in phd_locs if pd.notna(x)]

# research locations plotting 
phd_map = folium.Map()
#Loop through locations and add the markers on the map
for phd_location in range(len(phd_locs)): 
    folium.Marker([lat_list[phd_location], long_list[phd_location]],
                  popup=phd_locs[phd_location]).add_to(phd_map)

#display map    
#phd_map

In [10]:
# Masters locations 
lat_list = []
long_list = []
# Loop through rows to get the longitude and latitude of the PostDoc_locations (be aware of missing data)
for masters_location in df['masters_loc']:
   
    if pd.isnull(masters_location):
        lat = None
        long = None
    else:
        location_1 = geolocator.geocode(masters_location, timeout=10)
        if location_1 is None:
            raise ValueError("Geocode failed")
        lat = location_1.latitude
        long = location_1.longitude
    
    lat_list.append(lat)
    long_list.append(long)
    #print((lat,long,masters_location))
        
# #Saving lat and long in separate columns in the dataframe    
df['Masters_Latitude' ] = lat_list
df['Masters_Latitude'] = df['Masters_Latitude'].astype('float')

df['Masters_Longitude'] = long_list
df['Masters_Longitude'] = df['Masters_Longitude'].astype('float')
#df.head(40)

# create a map with research position locations
df_masters = df.loc[:,"Masters_Latitude":"Masters_Longitude"] #create a subset of df to deal with the na problem
df_masters = df_masters.dropna()
masters_locs = df.loc[:,"masters_loc"]
#print(df_undergrad)

lat_list = list(filter(None, lat_list))
long_list = list(filter(None, long_list))
masters_locs = [x for x in masters_locs if pd.notna(x)]

# research locations plotting 
masters_map = folium.Map()
#Loop through locations and add the markers on the map
for masters_location in range(len(masters_locs)): 
    folium.Marker([lat_list[masters_location], long_list[masters_location]],
                  popup=masters_locs[masters_location]).add_to(masters_map)

#display map    
#masters_map

In [11]:
# Post Doc locations 
lat_list = []
long_list = []
# Loop through rows to get the longitude and latitude of the PostDoc_locations (be aware of missing data)
for postdoc_location in df['postdoc_loc']:
   
    if pd.isnull(postdoc_location):
        lat = None
        long = None
    else:
        location_1 = geolocator.geocode(postdoc_location, timeout=10)
        if location_1 is None:
            raise ValueError("Geocode failed")
        lat = location_1.latitude
        long = location_1.longitude
    
    lat_list.append(lat)
    long_list.append(long)
    #print((lat,long,postdoc_location))
        
# #Saving lat and long in separate columns in the dataframe    
df['PostDoc_Latitude' ] = lat_list
df['PostDoc_Latitude'] = df['PostDoc_Latitude'].astype('float')

df['PostDoc_Longitude'] = long_list
df['PostDoc_Longitude'] = df['PostDoc_Longitude'].astype('float')
#df.head(40)

# create a map with research position locations
df_postdoc = df.loc[:,"PostDoc_Latitude":"PostDoc_Longitude"] #create a subset of df to deal with the na problem
df_postdoc = df_postdoc.dropna()
postdoc_locs = df.loc[:,"postdoc_loc"]
#print(df_undergrad)

lat_list = list(filter(None, lat_list))
long_list = list(filter(None, long_list))
postdoc_locs = [x for x in postdoc_locs if pd.notna(x)]

# research locations plotting 
postdoc_map = folium.Map()
#Loop through locations and add the markers on the map
for postdoc_location in range(len(postdoc_locs)): 
    folium.Marker([lat_list[postdoc_location], long_list[postdoc_location]],
                  popup=postdoc_locs[postdoc_location]).add_to(postdoc_map)

#display map    
#postdoc_map


In [12]:
# Faculty locations 
lat_list = []
long_list = []
# Loop through rows to get the longitude and latitude of the PostDoc_locations (be aware of missing data)
for faculty_location in df['faculty_loc']:
   
    if pd.isnull(faculty_location):
        lat = None
        long = None
    else:
        location_1 = geolocator.geocode(faculty_location, timeout=10)
        if location_1 is None:
            raise ValueError("Geocode failed")
        lat = location_1.latitude
        long = location_1.longitude
    
    lat_list.append(lat)
    long_list.append(long)
    #print((lat,long,faculty_location))
        
# #Saving lat and long in separate columns in the dataframe    
df['Faculty_Latitude' ] = lat_list
df['Faculty_Latitude'] = df['Faculty_Latitude'].astype('float')

df['Faculty_Longitude'] = long_list
df['Faculty_Longitude'] = df['Faculty_Longitude'].astype('float')
#df.head(40)

# create a map with research position locations
df_faculty = df.loc[:,"Faculty_Longitude":"Faculty_Latitude"] #create a subset of df to deal with the na problem
df_faculty = df_faculty.dropna()
faculty_locs = df.loc[:,"faculty_loc"]
#print(df_undergrad)

lat_list = list(filter(None, lat_list))
long_list = list(filter(None, long_list))
faculty_locs = [x for x in faculty_locs if pd.notna(x)]

# research locations plotting 
faculty_map = folium.Map()
#Loop through locations and add the markers on the map
for faculty_location in range(len(faculty_locs)): 
    folium.Marker([lat_list[faculty_location], long_list[faculty_location]],
                  popup=faculty_locs[faculty_location]).add_to(faculty_map)

#display map    
#faculty_map


In [13]:
# Create a new dataframe with just lat and long info 
loc_df = df.loc[:,'HometownLatitude':'Faculty_Longitude']
# View the dataframe 
#loc_df.head(20)

In [14]:
# create linear color gradiations (optional)
import matplotlib.pyplot as plt
rgba = plt.cm.autumn_r(np.linspace(0,1,7))
rgbb = plt.cm.winter_r(np.linspace(0,1,7))

rgb = rgbb[:, :3]
rgb255 = rgb * 255
rgb255 = rgb255.astype(int)
color = "rgb(%s, %s, %s)" % (rgb255[0][0], rgb255[0][1], rgb255[0][2])
color_1 = "rgb(%s, %s, %s)" % (rgb255[1][0], rgb255[1][1], rgb255[1][2])
color_2 = "rgb(%s, %s, %s)" % (rgb255[2][0], rgb255[2][1], rgb255[2][2])
color_3 = "rgb(%s, %s, %s)" % (rgb255[3][0], rgb255[3][1], rgb255[3][2])
color_4 = "rgb(%s, %s, %s)" % (rgb255[4][0], rgb255[5][1], rgb255[4][2])
color_5 = "rgb(%s, %s, %s)" % (rgb255[5][0], rgb255[5][1], rgb255[5][2])
color_6 = "rgb(%s, %s, %s)" % (rgb255[6][0], rgb255[6][1], rgb255[6][2])

In [16]:
m = Map(zoom=1, fitWorld=True)
rad = 5

#var circle = L.circle([click.lat, click.lng], 50000).addTo(map);

# hometown
hometown = []
for ii in range(len(locs)):
    coords = df.HometownLatitude[ii],df.HometownLongitude[ii]
    c = Circle()
    c.radius = rad
    c.location = coords
    c.color = "red"
    c.fill = True
    c.name = "Hometown location"
    hometown.append(c)
    ht_loc = HTML()
    ht_loc.value = locs[ii]
    c.popup = ht_loc
ht = lf.LayerGroup(name='hometown', layers=hometown)
m.add_layer(ht)

    
#ensure NA's are removed and turn undergrad_locs back into series  
undergrad_locs = [x for x in undergrad_locs if pd.notna(x)]
undergrad_locs = pd.Series(undergrad_locs)  
# undergrad
undergrad = []
for ii in range(len(undergrad_locs)):
    coords = df_undergrad.UndergradLatitude.iloc[ii],df_undergrad.UndergradLongitude.iloc[ii]
    c = Circle()
    c.radius = rad
    c.location = coords
    c.color = "purple"
    c.fill = True
    c.name = "Undergrad location"
    undergrad.append(c)
    underg_loc = HTML()
    underg_loc.value = undergrad_locs[ii]
    c.popup = underg_loc
ug = lf.LayerGroup(name='undergrad', layers=undergrad)
m.add_layer(ug)

#ensure NA's are removed and turn research_locs back into series 
research_locs = [x for x in research_locs if pd.notna(x)]
research_locs = pd.Series(research_locs) 
# plot research/lab manager positions
research = []
for ii in range(len(research_locs)):
    coords = df_research.RA_LM_Latitude.iloc[ii],df_research.RA_LM_Longitude.iloc[ii]
    c = Circle()
    c.radius = rad
    c.location = coords
    c.color = "pink"
    c.fill = True
    c.name = "RA location"
    research.append(c)
    research_loc = HTML()
    research_loc.value = research_locs[ii]
    c.popup = research_loc
ra = lf.LayerGroup(name='research', layers=research)
m.add_layer(ra)
    
#ensure NA's are removed and turn research_locs back into series 
masters_locs = [x for x in masters_locs if pd.notna(x)]
masters_locs = pd.Series(masters_locs)     
# plot master locations
master = []
for ii in range(len(masters_locs)):
    coords = df_masters.Masters_Latitude.iloc[ii],df_masters.Masters_Longitude.iloc[ii]
    c = Circle()
    c.radius = rad
    c.location = coords
    c.color = color
    c.fill = True
    c.name = "Master location"
    master.append(c)
    master_loc = HTML()
    master_loc.value = masters_locs[ii] 
    c.popup = master_loc
ms = lf.LayerGroup(name='master', layers=master)
m.add_layer(ms)

    
#ensure NA's are removed and turn phd_locs back into series 
phd_locs = [x for x in phd_locs if pd.notna(x)]
phd_locs = pd.Series(phd_locs)
# ensure df_phd is made without NA
df_phd = df.loc[:,"PHD_Latitude":"PHD_Longitude"] 
df_phd = df_phd.dropna()
# plot phd locations
phd = []
for ii in range(len(phd_locs)):
    coords = df_phd.PHD_Latitude.iloc[ii],df_phd.PHD_Longitude.iloc[ii]
    c = Circle()
    c.radius = rad
    c.location = coords
    c.color = color_4
    c.fill = True
    c.name = "PhD location"
    phd.append(c)
    phd_loc = HTML()
    phd_loc.value = phd_locs[ii]
    c.popup = phd_loc
ph = lf.LayerGroup(name='phd', layers=phd) 
m.add_layer(ph)

#ensure NA's are removed and turn postdoc_locs back into series 
postdoc_locs = [x for x in postdoc_locs if pd.notna(x)]
postdoc_locs = pd.Series(postdoc_locs) 
# ensure df_phd is made without NA
df_postdoc = df.loc[:,"PostDoc_Latitude":"PostDoc_Longitude"] 
df_postdoc = df_postdoc.dropna()
# plot postdoc level locations
postdoc = []
for ii in range(len(postdoc_locs)):
    coords = df_postdoc.PostDoc_Latitude.iloc[ii],df_postdoc.PostDoc_Longitude.iloc[ii]
    c = Circle()
    c.radius = rad
    c.location = coords
    c.color = "orange"
    c.fill = True
    c.name = "Postdoc location"
    postdoc.append(c)
    postdoc_loc = HTML()
    postdoc_loc.value = postdoc_locs[ii] 
    c.popup = postdoc_loc
psd = lf.LayerGroup(name='postdoc', layers=postdoc)
m.add_layer(psd)


#ensure NA's are removed and turn faculty_locs back into series     
faculty_locs = [x for x in faculty_locs if pd.notna(x)]
faculty_locs = pd.Series(faculty_locs)

# ensure df_faculty is made without NA
df_faculty = df.loc[:,"Faculty_Latitude":"Faculty_Longitude"] 
df_faculty = df_faculty.dropna()
# plot faculty level locations
faculty = []
for ii in range(len(faculty_locs)):
    coords = df_faculty.Faculty_Latitude.iloc[ii],df_faculty.Faculty_Longitude.iloc[ii]
    c = Circle()
    c.radius = rad
    c.location = coords
    c.color = "brown"
    c.fill = True
    c.name = "Faculty location"
    faculty.append(c)
    faculty_loc = HTML()
    faculty_loc.value = faculty_locs[ii]
    c.popup = faculty_loc
fc = lf.LayerGroup(name='faculty', layers=faculty)
m.add_layer(fc)

# add a zoom slider to your map
zoom_slider = IntSlider(description='Zoom level:', min=0, max=15, value=2)
jslink((zoom_slider, 'value'), (m, 'zoom'))
widget_control1 = WidgetControl(widget=zoom_slider, position='topright')
m.add_control(widget_control1)

m.add_control(FullScreenControl())
m.add_control(LayersControl())

#print map
m

Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map …